In [70]:
import pandas as pd
import numpy as np
from glob import glob

In [71]:
#Start off by reading the dir path and file names for each ticker csv
ticker_dir_list = glob("data/daily_20230216/*.csv")

In [86]:
ticker_dir = ticker_dir_list[0]
firstIndex = ticker_dir.rfind('\\') + 1
lastIndex = ticker_dir.rfind('.')
ticker = ticker_dir[firstIndex:lastIndex]

In [87]:
ticker_df = pd.read_csv(ticker_dir, parse_dates=['timestamp'])
ticker_df = ticker_df.rename(columns={'index':'timestamp', 
                          '1. open':'open',
                          '2. high':'high',
                          '3. low':'low',
                          '4. close':'close',
                          '5. adjusted close':'adjusted close',
                          '6. volume':'volume'}).drop(columns=['7. dividend amount', '8. split coefficient'])
                                                                                 
# Sort by date
ticker_df.sort_values(by=['timestamp'], inplace=True)  
# Reassign indices
ticker_df = ticker_df.reset_index().drop(columns=['index'])

In [88]:
# Find change in price for a day and a month
n = 1
ticker_df['price_delta_daily'] = ticker_df['adjusted close'].diff(n)
ticker_df['price_delta_monthly'] = ticker_df['adjusted close'].diff(n*30)

In [89]:
ticker_df['buy_signal_daily'] = ticker_df['price_delta_daily'].transform(np.sign)
ticker_df['buy_signal_monthly'] = ticker_df['price_delta_monthly'].transform(np.sign)

# Indicators

## Relative Strength Index (RSI)

$$RSI=100 - \frac{100}{1 + RS}$$

In [92]:
# Averaging Window for RSI     
n_rsi = 14

# Separate up and down days
up_df, down_df = ticker_df['price_delta_daily'].copy(), ticker_df['price_delta_daily'].copy()

up_df.loc['price_delta_daily'] = up_df.loc[(up_df['price_delta_daily'] < 0), 'price_delta_daily'] = 0


In [93]:
up_df

0                    0.000000
1                   -0.048911
2                    0.205427
3                   -0.007826
4                   -0.001956
                       ...   
3783                 0.030000
3784                 0.010500
3785                 0.069500
3786                 0.149500
price_delta_daily    0.000000
Name: price_delta_daily, Length: 3788, dtype: float64